In [13]:
import _scproxy
import pymssql
import pandas as pd
import numpy as np
import openpyxl
import warnings

warnings.filterwarnings('ignore')

In [14]:
def sql_connect(server='192.168.0.15',user='zhongxin_zyanbo',password='ZhangYB_068',sql=None):
    syntun_conn = pymssql.connect(server=server,
                            user=user,
                            password=password)
    syntun_cursor = syntun_conn.cursor()

    syntun_cursor.execute(sql)
    s = syntun_cursor.fetchall()
    syntun_cursor.close()
    syntun_conn.close()
    return s

In [15]:
d_=sql_connect(sql = 'select CAST ( 客户 AS nvarchar ( 500 ) ),	CAST ( 品类 AS nvarchar ( 500 ) ),	CAST ( 数据库名 AS nvarchar ( 500 ) ),	CAST ( 对比数据库名 AS nvarchar ( 500 ) ),	CAST ( 平台 AS nvarchar ( 500 ) ),	CAST ( 月份 AS nvarchar ( 500 ) ),	CAST ( 销售额 AS nvarchar ( 500 ) ),	CAST ( 升销量 AS nvarchar ( 500 ) ),	CAST ( 对比数据库平台 AS nvarchar ( 500 ) ),	CAST ( 对比数据库月份 AS nvarchar ( 500 ) ),	CAST ( 对比数据库销售额 AS nvarchar ( 500 ) ),	CAST ( 对比数据库升销量 AS nvarchar ( 500 ) )  from [QC].[dbo].历史对比配置')
QC_DF = pd.DataFrame(d_,columns =['客户','品类','数据库名','对比数据库名','平台','月份','销售额','升销量','对比数据库平台','对比数据库月份','对比数据库销售额','对比数据库升销量'])

In [16]:
columns_dict = {0:'平台名称',1:'品牌',2:'产品名称'}

In [17]:
# input('要运行的客户:')
# inp_ = '伊利,蒙牛'.split(',')
inp_ = '伊利'.split(',')
# input('请输入需要对比的库后缀,如果库的名称已配置完整此处可以为空:')
inp_date = ''

In [18]:
use_df_ = QC_DF[QC_DF['客户'].isin(inp_)]
use_df_

,客户,品类,数据库名,对比数据库名,平台,月份,销售额,升销量,对比数据库平台,对比数据库月份,对比数据库销售额,对比数据库升销量
1,伊利,常温液态奶,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,"D_C_PLATFORM,D_P_MANUFACTURE",D_T_MONTH,M_S_SALES,M_S_PHYSICALVOLUME,"D_C_PLATFORM,D_P_MANUFACTURE",D_T_MONTH,M_S_SALES,M_S_PHYSICALVOLUME


In [19]:
pd.set_option('display.float_format',lambda x : '%.2f' % x)


In [20]:
a_box,b_box,c_box = [pd.DataFrame()],[pd.DataFrame()],[pd.DataFrame()]
def lg(n):
    use_df = use_df_[n:n+1]
    sjk = use_df[['数据库名','平台','月份','销售额','升销量']]
    db_sjk = use_df[['对比数据库名','对比数据库平台','对比数据库月份','对比数据库销售额','对比数据库升销量']]
    sjk_ = pd.concat([sjk[['数据库名','月份','销售额','升销量']],sjk['平台'].str.split(',',expand=True).rename(columns = columns_dict)],axis=1)
    db_sjk_ = pd.concat([db_sjk[['对比数据库名','对比数据库月份','对比数据库销售额','对比数据库升销量']],db_sjk['对比数据库平台'].str.split(',',expand=True).rename(columns = columns_dict)],axis=1)

    sql_li_a = []
    sql_li_b = []
    for i in sjk_.columns:
        sql_li_a.append( sjk_[i].values[0])
    for i in db_sjk_.columns:
        sql_li_b.append(db_sjk_[i].values[0])
    
    try:
        s,a,b,c,d,e,f = sql_li_a
        S,A,B,C,D,E,F = sql_li_b
        S = S+inp_date
    except:
        try:
            s,a,b,c,d,e = sql_li_a
            S,A,B,C,D,E = sql_li_b
            S = S+inp_date
        except:
            s,a,b,c,d = sql_li_a
            S,A,B,C,D = sql_li_b
            S = S+inp_date
    
    
    
    if len(sjk['平台'].str.split(',')) >= 0:
        # a,b,c,d = sql_li_a
        a_sql = f"SELECT {a},CAST( {d} AS nvarchar ( 500 ) ),sum({b}),sum({c}) from {s} group by {a},{d}"
        
        # A,B,C,D = sql_li_b
        b_sql = f"SELECT {A},CAST( {D} AS nvarchar ( 500 ) ),sum({B}),sum({C}) from {S} group by {A},{D}"
        
        
        a_df = pd.DataFrame(sql_connect(sql=a_sql),columns=['月份','平台','销售额','升销量'])
        b_df = pd.DataFrame(sql_connect(sql=b_sql),columns=['月份','平台','销售额_db','升销量_db'])
        c_df = a_df.merge(b_df,how='left',on=['月份','平台']).assign(
            数据库名 = s,
            数据库名_db = S,
            销售额差异 = lambda x: x['销售额']-x['销售额_db'],
            升销量差异 = lambda x: x['升销量']-x['升销量_db'],
            是否差异 = lambda x: abs(x['销售额差异']+x['升销量差异']) > 0000.1)
        
        a_box.append(c_df[c_df['是否差异'] == True])
        
    if len(sjk['平台'].str.split(',')) >= 1:
        # a,b,c,d,e = sql_li_a
        a_sql = f"SELECT {a},CAST( {d} AS nvarchar ( 500 ) ),CAST( {e} AS nvarchar ( 500 ) ),sum({b}),sum({c}) from {s} group by {a},{d},{e}"
        
        # A,B,C,D,E = sql_li_b
        b_sql = f"SELECT {A},CAST( {D} AS nvarchar ( 500 ) ),CAST( {E} AS nvarchar ( 500 ) ),sum({B}),sum({C}) from {S} group by {A},{D},{E}"
        
        a_df = pd.DataFrame(sql_connect(sql=a_sql),columns=['月份','平台','品牌','销售额','升销量'])
        b_df = pd.DataFrame(sql_connect(sql=b_sql),columns=['月份','平台','品牌','销售额_db','升销量_db'])
        c_df = a_df.merge(b_df,how='left',on=['月份','平台','品牌']).assign(
            数据库名 = s,
            数据库名_db = S,
            销售额差异 = lambda x: x['销售额']-x['销售额_db'],
            升销量差异 = lambda x: x['升销量']-x['升销量_db']
            ,
            是否差异 = lambda x: abs(x['销售额差异']+x['升销量差异']) > 0
            )
        
        b_box.append(c_df[c_df['是否差异'] == True])
    
    if len(sjk['平台'].str.split(',')) >= 2:
        # a,b,c,d,e,f = sql_li_a
        a_sql = f"SELECT {a},CAST( {d} AS nvarchar ( 500 ) ),CAST( {e} AS nvarchar ( 500 ) ),CAST( {f} AS nvarchar ( 500 ) ),sum({b}),sum({c}) from {s} group by {a},{d},{e},{f}"
        
        # A,B,C,D,E,F = sql_li_b
        b_sql = f"SELECT {A},CAST( {D} AS nvarchar ( 500 ) ),CAST( {E} AS nvarchar ( 500 ) ),CAST( {F} AS nvarchar ( 500 ) ),sum({B}),sum({C}) from {S} group by {A},{D},{E},{F}"
        
        a_df = pd.DataFrame(sql_connect(sql=a_sql),columns=['月份','平台','品牌','销售额','升销量'])
        b_df = pd.DataFrame(sql_connect(sql=b_sql),columns=['月份','平台','品牌','销售额_db','升销量_db'])
        c_df = a_df.merge(b_df,how='left',on=['月份','平台','品牌']).assign(
            数据库名 = s,
            数据库名_db = S,
            销售额差异 = lambda x: x['销售额']-x['销售额_db'],
            升销量差异 = lambda x: x['升销量']-x['升销量_db'],
            是否差异 = lambda x: abs(x['销售额差异']+x['升销量差异']) > 0000.1)

        c_box.append(c_df[c_df['是否差异'] == True])

    # sjk_len = len(sjk_.columns[~sjk_.columns.isin(['数据库名','月份','销售额','升销量'])])
    
    
    # sql = 
    # sql_ = 
    
    return a_box,b_box,c_box
    return len(sjk['平台'].str.split(','))
    


In [21]:
for i in range(len(use_df_)):
    lg(i)

In [22]:
pd.concat(b_box,axis=0).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26886 entries, 4 to 96887
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   月份       26886 non-null  object 
 1   平台       26886 non-null  object 
 2   品牌       26886 non-null  object 
 3   销售额      26886 non-null  float64
 4   升销量      26886 non-null  float64
 5   销售额_db   26886 non-null  float64
 6   升销量_db   26886 non-null  float64
 7   数据库名     26886 non-null  object 
 8   数据库名_db  26886 non-null  object 
 9   销售额差异    26886 non-null  float64
 10  升销量差异    26886 non-null  float64
 11  是否差异     26886 non-null  bool   
dtypes: bool(1), float64(6), object(5)
memory usage: 2.5+ MB


In [23]:
pd.concat(b_box,axis=0)

,月份,平台,品牌,销售额,升销量,销售额_db,升销量_db,数据库名,数据库名_db,销售额差异,升销量差异,是否差异
4,201912,拼多多,夏进,24.97,21688.94,24.97,21688.94,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,0.00,0.00,True
5,202209,天猫,达利园,666.58,791723.00,666.58,791723.00,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,0.00,0.00,True
6,202011,京东,维素,0.42,137.00,0.42,137.00,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,0.00,0.00,True
7,202012,拼多多,益生元,3.13,8804.64,3.13,8804.64,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,0.00,0.00,True
12,202101,天猫,牛益牛,0.16,142.00,0.16,142.00,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,-0.00,0.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...
96875,201705,京东,维纯,198.82,180779.00,198.82,180779.00,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,-0.00,0.00,True
96878,201710,京东,国农,1.67,536.37,1.67,536.37,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,-0.00,0.00,True
96883,202204,天猫,展艺,6.77,2260.90,6.77,2260.90,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,-0.00,0.00,True
96884,202001,京东,阿尔发,0.45,266.40,0.45,266.40,[send_out].[dbo].[YILI_DATA_YTN_E_MONTH_NEW],excellent.dbo.YILI_DATA_YTN_E_MONTH_NEW_202210,0.00,0.00,True


In [24]:
# a_box,b_box,c_box
import openpyxl
from openpyxl import load_workbook
with pd.ExcelWriter(f'结果/{inp_}月报对比_结果.xlsx') as mn_writer:
    pd.concat(a_box,axis=0).to_excel(mn_writer,sheet_name='平台',na_rep='',index=False)
    pd.concat(b_box,axis=0).to_excel(mn_writer,sheet_name='平台 品牌',na_rep='',index=False)
    pd.concat(c_box,axis=0).to_excel(mn_writer,sheet_name='平台 品牌 产品名称',na_rep='',index=False)
